In [0]:
import numpy as np
import time
import cv2
import os
os.mkdir("weights")
!wget -O data https://pjreddie.com/media/files/yolov3.weights -P weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg -P weights

In [0]:
# loading label
LABELS = open("coco.names").read().strip().split("\n")

# colors to represent each possible class labe
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

# paths to the YOLO weights and model configuration
weightsPath = os.path.join("weights", "yolov3.weights")
configPath = os.path.join("weights", "yolov3.cfg")

# loading YOLO object detector trained on 80 classes
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# load input image 
image = cv2.imread("driver.png")
(H, W) = image.shape[:2]

# the output layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)


# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []

# loop over each of the layer outputs
for output in layerOutputs:
	# loop over each of the detections
	for detection in output:
		
		scores = detection[5:]
		classID = np.argmax(scores)
		confidence = scores[classID]

		
		# confidence is greater than the minimum threshold of 0.5
		if confidence > 0.5: 
			# yolo returns the center (x, y)-coordinates of the bounding
			# box followed by the boxes' width and height
			box = detection[0:4] * np.array([W, H, W, H])
			(centerX, centerY, width, height) = box.astype("int")

			# use the center (x, y)-coordinates to derive the top and
			# and left corner of the bounding box
			x = int(centerX - (width / 2))
			y = int(centerY - (height / 2))

			
			boxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)

# apply non-maxima suppression to suppress weak, overlapping bounding box
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.3,
	0.3)

# ensuring at least one detection exists
if len(idxs) > 0:
	# loop over the indexes we are keeping
	for i in idxs.flatten():
		# extract the bounding box coordinates
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])

		# draw a bounding box rectangle and label on the image
		color = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, color, 2)

# show the output image
cv2.imshow(image)